*Note: In this workbook, we try to replicate the results from the classic paper "Talk of the Network: A Complex Systems Look at the Underlying Process of Word-of-Mouth", Goldenberg, Libai and Muller (2001). This is a self-didactic attempt.*

In [1]:
using LightGraphs
using Distributions, DataFrames, GLM, ProgressMeter

INFO: Precompiling module ProgressMeter.


In [2]:
srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.97231, 1.59736, 1.68151, 1.49362, 1.39143, 1.61799, 1.58948, 1.78774, 1.13809, 1.19662  …  1.92824, 1.20716, 1.7229, 1.01022, 1.84855, 1.56883, 1.88421, 1.16472, 1.42936, 1.58411], 382)

# 1. Introduction 

In [Talk of the Network](https://www0.gsb.columbia.edu/mygsb/faculty/research/pubfiles/3391/TalkofNetworks.pdf), the authors  explore the pattern of personal communication between an individual's core friends group (strong ties) and a wider set of acquaintances (weak ties). This remarkable study is one of the first ones in marketing that explored the influence of social networks on the diffusion of marketing messages. The key questions investigated in this paper are:

- What matters more - strong ties or weak ties?
- What effect does the size of an average individuals network have?
- How does advertising interact with the diffusion through weak ties and that through strong ties

In this workbook, we focus on replicating the efforts of the authors to answer the first question: do strong ties or weak ties influence the speed of information dissemination in a network?

# 2. Initializing the network

This study employs a large number of synthetic networks as substrates to study the diffusion of information diffusion. To quote the authors logic to create and initialize the networks:

> *"Each individual belongs to a single personal network. Each network consists of individuals who are connected by strong ties. In each period, individuals also conduct a finte number of weak tie interactions outside their personal networks... We divide the entire market equally into personal networks, in which each individual can belong to one network. In addition, in each period, every individual conducts random meetings with individuals external to his personal network."*

Given this specification, we utilize the built-in complete graph generator from [LightGraphs](http://juliagraphs.github.io/LightGraphs.jl/latest/generators.html#Graph-Generators-1) to build several mini-regular networks and then allow individuals in each of these mini-networks to mingle. Our final data structure is hence a dictionary of several complete networks that are built based on the number of strong ties for each individual. Note that each individual in the network has a fixed number of strong ties ($s$) and weak ties ($w$).

In [3]:
function initialize_network(n_nodes::Int, n_strong_ties::Int)
    G = Dict{Int, LightGraphs.SimpleGraphs.SimpleGraph}()

    for g in 1:floor(Int, n_nodes/n_strong_ties)
        G[g] = CompleteGraph(n_strong_ties)
    end

    return G
end

initialize_network (generic function with 1 method)

# 3. Model

## 3.1 Assumptions

The probability of activation of a node, i.e., an uninformed individual turning to informed can happen in three ways: through a strong tie with probability $\beta_s$, through a weak tie with probability $\beta_w$ or through external marketing efforts with probability $\alpha$. In line with conventional wisdom, the authors assume $\alpha < \beta_w < \beta_s$. 

At timestep $t$, if an individual is connected to $m$ strong ties and $j$ weak ties, the probability of the individual being informed in this time step is:

$$
p(t) = 1 - (1- \alpha)(1 - \beta_w)^j(1 - \beta_s)^m
$$

The outcome variable of interest is the number of time steps elapsed till 95% of the network engages.

## 3.2 Execution

Following our earlier discussion on the construction of substrate networks, each node in the network belongs to a complete sub-network. In addition, at each time step each node interacts with a fixed number of weak ties chosen at random from sub-networks other than its own.

*Step 1:* At $t = 0$, the status of all nodes is set to `false`

*Step 2:* For each node, the probability $p(t)$ of being informed is calculated using the above equation. A random draw $U$ is made from a standard uniform distribution and compared with $p(t)$. If $U < p(t)$ the status of the node is changed to `true`

*Step 3:* In each successive time step, Step 2 is repeated till 95% of the total network (of size 3000) engages

We now look at several helper functions that execute the above logic

### 3.2.1 Reset node status

The node status is stored as a dictionary of `BitVector`'s. At the beginning of each simulation run, we call the following function to set the status of all the nodes to `false`. 

In [4]:
function reset_node_status(G::Dict{Int, LightGraphs.SimpleGraphs.SimpleGraph})

    node_status = Dict{Int, BitVector}()

    for key in keys(G)
        node_status[key] = falses(nv(G[key]))
    end

    return node_status
end

reset_node_status (generic function with 1 method)

### 3.2.2 Updating status of the nodes

At each time step, we execute two tasks. First, we allow the nodes to mingle randomly with their strong ties and with weak ties from other sub-networks. At this point, we count the number of active strong and weak ties for each node. Then, we use this information to update the status of all the nodes in the network.

The first function executes the "random meetings" with weak ties as discussed in the paper. For each node we generate a random sample (without replacement) of size $w$ from sub-networks other than its own. We then count the number of active ties in its own sub-network and among the random sample taken from the rest of the network.

In [5]:
function random_meetings(G::Dict{Int,LightGraphs.SimpleGraphs.SimpleGraph},
                         node::Int,
                         node_network_id::Int,
                         node_status::Dict{Int, BitVector},
                         n_weak_ties::Int)

    n_active_str_ties, n_active_wk_ties = 0, 0

    # Count number of active strong ties of the node by querying their status

    for nbr in neighbors(G[node_network_id], node)
        if node_status[node_network_id][nbr] == true
            n_active_str_ties += 1
        end
    end

    # Choose a random sample of size `n_weak_ties` from the other sub-networks and query
    # their status. We first sample the network id, and use this to sample a random node
    # in the sub-network defined by this id.

    all_network_ids = collect(keys(G))

    other_network_ids = all_network_ids[all_network_ids .!= node_network_id]
    possible_weak_ties = []
    nsamples = 1

    while nsamples < n_weak_ties
        rand_network_id = sample(other_network_ids)
        rand_nbr = sample(vertices(G[rand_network_id]))
        if !((rand_network_id, rand_nbr) in possible_weak_ties)
            push!(possible_weak_ties, (rand_network_id, rand_nbr))
            nsamples += 1
        end
    end

    for (network_id, weak_tie) in possible_weak_ties
        if node_status[network_id][weak_tie] == true
            n_active_wk_ties += 1
        end
    end

    return n_active_str_ties, n_active_wk_ties
end

random_meetings (generic function with 1 method)

The second function conducts the updation of the status of all the nodes at each time step by calculating the probability of activation. 

In [6]:
function update_status!(G::Dict{Int,LightGraphs.SimpleGraphs.SimpleGraph},
                        node_status::Dict{Int, BitVector},
                        n_weak_ties::Int,
                        alpha::Float64, beta_w::Float64, beta_s::Float64)

    # assuming that the nodes update in random order

    for node_network_id in shuffle(collect(keys(G)))
        for node in shuffle(vertices(G[node_network_id]))
            n_active_str_ties, n_active_wk_ties = random_meetings(G, node, node_network_id, node_status, n_weak_ties)
            activation_prob = 1 - (1 - alpha) * (1 - beta_w)^n_active_wk_ties * (1 - beta_s)^n_active_str_ties

            if rand(Uniform()) < activation_prob
                node_status[node_network_id][node] = true
            end
        end
    end

    return nothing
end

update_status! (generic function with 1 method)

### 3.2.4 Simulation on the parameter space

The function `execute_simulation` puts together the scaffolding to set up the parameter space $(s, w, \alpha, \beta_w, \beta_s)$ and execute diffusion along the network. From what I can gather from the paper, one simulation was carried out at each point on the parameter space. No further details regarding the execution are mentioned except that since each parameter has 7 levels, a total of $7^5 = 16,807$ simulations were executed in a factorial design. In this workbook, we work on a smaller parameter space using 3 levels for each parameter.

Also, I am assuming that the network is drawn at random for each run of the simulation.

One more interesting thing to note: The authors mention that their simulations were written in C, it would be interesting to compare the execution times with Julia. This is a non-standard problem that tests both the robustness of Julia types and its execution speed (maybe this will prompt someone to make a pull request!).

In [7]:
println("Number of strong ties per node (s): ", floor.(Int, linspace(5, 29, 3)))
println("Number of weak ties per node(w): ", floor.(Int, linspace(5, 29, 3)))
println("Effect of advertising (α): ", collect(linspace(0.0005, 0.01, 3)))
println("Effect of weak ties (β_w): ", collect(linspace(0.005, 0.015, 3)))
println("Effect of strong ties (β_s): ", collect(linspace(0.01, 0.07, 3)))

Number of strong ties per node (s): [5, 17, 29]
Number of weak ties per node(w): [5, 17, 29]
Effect of advertising (α): [0.0005, 0.00525, 0.01]
Effect of weak ties (β_w): [0.005, 0.01, 0.015]
Effect of strong ties (β_s): [0.01, 0.04, 0.07]


In [8]:
parameter_space = [(s, w, alpha, beta_w, beta_s) for s in floor.(Int, linspace(5, 29, 3)), 
                                                     w in floor.(Int, linspace(5, 29, 3)),
                                                     alpha in linspace(0.0005, 0.01, 3),
                                                     beta_w in linspace(0.005, 0.015, 3),
                                                     beta_s in linspace(0.01, 0.07, 3)]

size(parameter_space), length(parameter_space)

((3, 3, 3, 3, 3), 243)

In [11]:
function execute_simulation(parameter_space, n_nodes::Int)

    # n_nodes dictates how big the network will be
    # We cannot pre-allocate the output since we do not know for how many time steps the simulation will
    # run at each setting

    output = DataFrame(s = Int[], w = Int[], alpha = Float64[],
                       beta_w = Float64[], beta_s = Float64[],
                       t = Int[], num_engaged = Int[])
    
    println("Beginning simulation at : ", Dates.format(now(), "HH:MM"))
    println("You might want to grab a cup of coffee while Julia brews the simulation...")

    @showprogress 1 "Crunching numbers while you munch..." for (s, w, alpha, beta_w, beta_s) in parameter_space[1:end]

        G = initialize_network(n_nodes, s)

        node_status = reset_node_status(G)

        num_engaged = sum(sum(values(node_status)))

        t = 1

        # Continue updates at each setting till 95% of the network engages

        while num_engaged < floor(Int, 0.95 * n_nodes)
            update_status!(G, node_status, w, alpha, beta_w, beta_s)
            num_engaged = sum(sum(values(node_status)))
            push!(output, [s, w, alpha, beta_w, beta_s, t, num_engaged])
            t += 1
        end
    end
    
    return output
end

execute_simulation (generic function with 1 method)

In [13]:
results = execute_simulation(parameter_space, 3000)

Beginning simulation at : 15:13
You might want to grab a cup of coffee while Julia brews the simulation...


Crunching numbers while you munch...100%|███████████████| Time: 0:07:27


,s,w,alpha,beta_w,beta_s,t,num_engaged
1,5,5,0.0005,0.005,0.01,1,1
2,5,5,0.0005,0.005,0.01,2,2
3,5,5,0.0005,0.005,0.01,3,3
4,5,5,0.0005,0.005,0.01,4,4
5,5,5,0.0005,0.005,0.01,5,8
6,5,5,0.0005,0.005,0.01,6,11
7,5,5,0.0005,0.005,0.01,7,16
8,5,5,0.0005,0.005,0.01,8,21
9,5,5,0.0005,0.005,0.01,9,24
10,5,5,0.0005,0.005,0.01,10,27


# 4. Discussion

To answer the research questions, the authors resort to simple linear regression. 

Since our focus in this workbook is on highlighting the strengths of the JuliaGraphs ecosystem, we keep the regression modeling at the most basic level.

As discussed earlier, the outcome is the time taken for 95% of the network to engage with the message. The features used to predict this outcome are $s$, $w$, $\alpha$, $\beta_w$ and $\beta_S$. 

In [14]:
head(results)

,s,w,alpha,beta_w,beta_s,t,num_engaged
1,5,5,0.0005,0.005,0.01,1,1
2,5,5,0.0005,0.005,0.01,2,2
3,5,5,0.0005,0.005,0.01,3,3
4,5,5,0.0005,0.005,0.01,4,4
5,5,5,0.0005,0.005,0.01,5,8
6,5,5,0.0005,0.005,0.01,6,11


To build the data required for the linear modeling, we group the data by each parameter setting and calculate the time the network takes to reach 95% activation.

In [15]:
all_engaged = by(results, [:s, :w, :alpha, :beta_w, :beta_s], df -> DataFrame(T95 = maximum(df[:t])));
head(all_engaged)

,s,w,alpha,beta_w,beta_s,T95
1,5,5,0.0005,0.005,0.01,153
2,5,5,0.0005,0.005,0.04,102
3,5,5,0.0005,0.005,0.07,87
4,5,5,0.0005,0.01,0.01,112
5,5,5,0.0005,0.01,0.04,71
6,5,5,0.0005,0.01,0.07,57


We then run a simple linear model on the data

In [17]:
ols = lm(@formula(T95 ~ s + w + alpha + beta_s + beta_w), all_engaged)

DataFrames.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: T95 ~ 1 + s + w + alpha + beta_s + beta_w

Coefficients:
              Estimate Std.Error  t value Pr(>|t|)
(Intercept)    84.8721   3.03928   27.925   <1e-76
s              -1.0144 0.0741088  -13.688   <1e-31
w            -0.820988 0.0741088 -11.0781   <1e-22
alpha         -1394.41   187.222  -7.4479   <1e-11
beta_s          -300.0   29.6435 -10.1203   <1e-19
beta_w        -1107.41   177.861 -6.22625    <1e-8


In [18]:
r2(ols)

0.6813421157280519

This is a rather strong finding. The speed of information diffusion is impacted equally strongly by both strong ties and weak ties. As the authors note, the surprising aspect of this strudy is that the effect of weak ties is rather strong despite the inferiority of the weak ties parameter in the model assumptions.